<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Red%20convolucional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [364]:
import numpy as np
import tensorflow as tf
import pandas as pd
import tensorflow.keras as keras
import sklearn as sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Conv1D
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras import layers
from google.colab import files
from sklearn.metrics import roc_auc_score
from sklearn import metrics
!pip install talos
import talos

In [365]:
# CARGAMOS LOS CSV
url_train = 'https://raw.githubusercontent.com/JulioCastillo1/TP1-Datos/master/train.csv'
url_test = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test.csv'

train_df = pd.read_csv(url_train)
test_df = pd.read_csv(url_test)
train_df = train_df.drop(['keyword', 'location','id'], axis = 1)
train_df.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [366]:
train_df.shape

(7613, 2)

In [367]:
x_train, x_validation, y_train, y_validation = train_test_split(train_df['text'],train_df['target'],random_state=7,test_size = 0.20)
x_train = train_df['text']
y_train = train_df['target']

In [368]:
def modelardo(x_train,y_train,x_test,y_test,params):
  vectorizer = CountVectorizer()
  vectorizer.fit(x_train)

  x_train = vectorizer.transform(x_train)
  x_test = vectorizer.transform(x_test)
  input_dim = x_train.shape[1]
  model = Sequential()
  model.add(layers.Dense(params['first neuron units'],input_dim=input_dim,activation=params['first neuron activation']))
  model.add(layers.Dense(1,activation = 'sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  out = model.fit(x_train,y_train,epochs = params['epochs'],verbose = 0,validation_data = (x_test,y_test),batch_size=params['batch size'])
  return out,model


In [369]:
x_train.shape
#t = talos.Scan(x_train,y_train,params=p,model=modelardo,experiment_name='diabetes')
#Este proceso tarda 1,5hs aprox en colab. Y tiene muy poquitos hiperparámetros para tunear

(7613,)

In [370]:
#experiment_data = t.data
#experiment_data.head()

In [371]:
#experiment_data['val_accuracy'].idxmax()

In [372]:
#experiment_data.iloc[205]

In [373]:
#vectorizer = CountVectorizer()
#vectorizer.fit(train_df['text'])

#data_train = vectorizer.fit_transform(train_df['text']).toarray()


In [374]:
# Función con el modelo para pasar por parámetro a talos.

def modelo(x_train,y_train,x_test,y_test,params):
  tokenizer = Tokenizer(num_words=2000)
  tokenizer.fit_on_texts(x_train)

  train_sequences = tokenizer.texts_to_sequences(x_train)
  validation_sequences = tokenizer.texts_to_sequences(x_validation)
  train_data = pad_sequences(train_sequences, maxlen=100)
  validation_data = pad_sequences(validation_sequences, maxlen=100)


  #input_dim = x_train.shape[1]

  best_model = Sequential()
  best_model.add(layers.Embedding(2000, 100, input_length=100))
  best_model.add(Conv1D(params['neuron_units_1'], 5, activation = params['activation']))
  best_model.add(layers.GlobalAveragePooling1D())
  #best_model.add(Conv1D(512, 5, activation='relu'))
  #best_model.add(layers.GlobalAveragePooling1D())
  #best_model.add(Conv1D(512, 5, activation='relu'))
  #best_model.add(layers.GlobalAveragePooling1D())
  #best_model.add(MaxPooling1D(5))
  #best_model.add(layers.Dropout(0.25))
  #best_model.add(Flatten())
  #best_model.add(LSTM(64))
  best_model.add(layers.Dense(params['neuron_units_1'], activation = params['activation']))
  #best_model.add(layers.Dense(128 ,activation='relu'))
  best_model.add(layers.Dense(units = 1, activation='sigmoid'))
  #best_model.add(layers.Dense(1,activation = 'sigmoid'))
  best_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  out = best_model.fit(train_data,y_train,epochs = params['epochs'],verbose = 1,validation_data = (validation_data,y_validation),batch_size=params['batch size'])
  return out, best_model

#loss, accuracy = best_model.evaluate(train_data, y_train, verbose=False)
#print("Training Accuracy: {:.4f}".format(accuracy))
#loss, accuracy = best_model.evaluate(validation_data, y_validation, verbose=False)
#print("Testing Accuracy:  {:.4f}".format(accuracy))

p = {
    'neuron_units_1': [20,32,64,128],
    'activation' : ['relu','elu','softmax','tanh','exponential'],
    'epochs' : [1,5,10],
    'batch size': [5,100,5],
}

#t = talos.Scan(x_train, y_train, params = p, model = modelo, experiment_name = 'Convo hyperparam tuning')


  0%|          | 0/180 [00:00<?, ?it/s]

Train on 5329 samples, validate on 1523 samples
Epoch 1/1
5329/5329 [==============================] - 11s 2ms/step - loss: 0.5344 - accuracy: 0.7211 - val_loss: 0.4146 - val_accuracy: 0.8181



  1%|          | 1/180 [00:15<45:15, 15.17s/it]

Train on 5329 samples, validate on 1523 samples
Epoch 1/1
5329/5329 [==============================] - 10s 2ms/step - loss: 0.5261 - accuracy: 0.7399 - val_loss: 0.3952 - val_accuracy: 0.8352



  1%|          | 2/180 [00:26<41:13, 13.90s/it]

...


 99%|█████████▉| 178/180 [2:18:34<02:29, 74.88s/it]

Train on 5329 samples, validate on 1523 samples
Epoch 1/10
5329/5329 [==============================] - 11s 2ms/step - loss: 0.9131 - accuracy: 0.6887 - val_loss: 0.4275 - val_accuracy: 0.8168
Epoch 2/10
5329/5329 [==============================] - 11s 2ms/step - loss: 0.4163 - accuracy: 0.8219 - val_loss: nan - val_accuracy: 0.8345
Epoch 3/10
5329/5329 [==============================] - 11s 2ms/step - loss: nan - accuracy: 0.0595 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/10
5329/5329 [==============================] - 11s 2ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/10
5329/5329 [==============================] - 11s 2ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 6/10
5329/5329 [==============================] - 11s 2ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 7/10
5329/5329 [==============================] - 11s 2ms/step - loss: nan 


 99%|█████████▉| 179/180 [2:20:26<01:25, 85.83s/it]

Train on 5329 samples, validate on 1523 samples
Epoch 1/10
5329/5329 [==============================] - 13s 2ms/step - loss: 3.6565 - accuracy: 0.6767 - val_loss: 0.4549 - val_accuracy: 0.8043
Epoch 2/10
5329/5329 [==============================] - 12s 2ms/step - loss: 0.4193 - accuracy: 0.8176 - val_loss: 0.3984 - val_accuracy: 0.8227
Epoch 3/10
5329/5329 [==============================] - 13s 2ms/step - loss: nan - accuracy: 0.4586 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/10
5329/5329 [==============================] - 13s 2ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/10
5329/5329 [==============================] - 13s 2ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 6/10
5329/5329 [==============================] - 14s 3ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 7/10
5329/5329 [==============================] - 14s 3ms/step - loss: n


100%|██████████| 180/180 [2:22:39<00:00, 47.55s/it] 


In [375]:
#experiment_data = t.data
#experiment_data.head()

,round_epochs,val_loss,val_accuracy,loss,accuracy,activation,batch size,epochs,neuron_units_1
0,1,0.414556,0.818122,0.534374,0.721148,relu,5,1,20
1,1,0.395249,0.835194,0.526109,0.739914,relu,5,1,32
2,1,0.402230,0.832567,0.515658,0.747607,relu,5,1,64
3,1,0.383480,0.845699,0.510647,0.748170,relu,5,1,128
4,5,0.330322,0.866054,0.259439,0.895478,relu,5,5,20


In [376]:
#maxim=experiment_data['val_accuracy'].idxmax()

In [377]:
#experiment_data.iloc[maxim]

round_epochs            10
val_loss          0.273535
val_accuracy      0.911359
loss              0.144524
accuracy          0.945393
activation            tanh
batch size               5
epochs                  10
neuron_units_1          32
Name: 141, dtype: object

In [378]:
experiment_data.to_csv('experiment.csv')
#files.download('experiment.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Dalos experimentales:

round_epochs            10
val_loss          0.273535
val_accuracy      0.911359
loss              0.144524
accuracy          0.945393
activation            tanh
batch size               5
epochs                  10
neuron_units_1          32





In [379]:
# Divide el set en train y validation, y entrena con el modelo tomando los hiperparámetros sugeridos

x_train, x_test, y_train, y_test = train_test_split(train_df['text'],train_df['target'],random_state=7,test_size = 0.20)
  
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(x_train)

train_sequences = tokenizer.texts_to_sequences(x_train)
validation_sequences = tokenizer.texts_to_sequences(x_validation)
train_data = pad_sequences(train_sequences, maxlen=100)
validation_data = pad_sequences(validation_sequences, maxlen=100)


#input_dim = x_train.shape[1]

best_model = Sequential()
best_model.add(layers.Embedding(2000, 100, input_length=100))
best_model.add(Conv1D(128, 5, activation='elu'))
best_model.add(layers.GlobalAveragePooling1D())
#best_model.add(Conv1D(512, 5, activation='elu'))
#best_model.add(layers.GlobalAveragePooling1D())
#best_model.add(Conv1D(512, 5, activation='elu'))
#best_model.add(layers.GlobalAveragePooling1D())
#best_model.add(MaxPooling1D(5))
#best_model.add(layers.Dropout(0.25))
#best_model.add(Flatten())
best_model.add(layers.Dense(128 ,activation='elu'))
#best_model.add(layers.Dense(128 ,activation='relu'))
best_model.add(layers.Dense(1,activation='sigmoid'))
best_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
out = best_model.fit(train_data,y_train,epochs=5,verbose = 1,validation_data = (validation_data,y_validation),batch_size=50)

Train on 6090 samples, validate on 1523 samples
Epoch 1/5
6090/6090 [==============================] - 8s 1ms/step - loss: 0.6434 - accuracy: 0.6197 - val_loss: 0.5679 - val_accuracy: 0.7255
Epoch 2/5
6090/6090 [==============================] - 8s 1ms/step - loss: 0.4508 - accuracy: 0.7944 - val_loss: 0.4634 - val_accuracy: 0.7958
Epoch 3/5
6090/6090 [==============================] - 7s 1ms/step - loss: 0.3752 - accuracy: 0.8401 - val_loss: 0.4552 - val_accuracy: 0.8056
Epoch 4/5
6090/6090 [==============================] - 8s 1ms/step - loss: 0.3429 - accuracy: 0.8509 - val_loss: 0.4671 - val_accuracy: 0.7925
Epoch 5/5
6090/6090 [==============================] - 8s 1ms/step - loss: 0.3379 - accuracy: 0.8529 - val_loss: 0.4924 - val_accuracy: 0.7787


In [380]:
prediction=best_model.predict_classes(validation_data, batch_size=50)
score = roc_auc_score(y_validation, prediction)
score

0.781778196504451

In [381]:
accuracy=metrics.accuracy_score(y_validation, prediction)
accuracy

0.7787261982928431

In [382]:
#Reentrena con el set completo y realiza predicción.

#set_sequences = tokenizer.texts_to_sequences(train_df['text'])
#set_data = pad_sequences(set_sequences, maxlen=100)

#out = best_model.fit(set_data,train_df['target'],epochs = 5,verbose = 1,batch_size=62)

#out = best_model.fit(data_train,train_df['target'],epochs = 5,verbose = 1,batch_size=62)
#data_test = vectorizer.transform(test_df['text']).toarray()
test_prediction = tokenizer.texts_to_sequences(test_df['text'])
prediction_data = pad_sequences(test_prediction, maxlen=100)

prediction2=best_model.predict_classes(prediction_data, batch_size=50)
prediction2

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

In [383]:
# Genera archivo submit.

test_df['target'] = prediction2
export = pd.DataFrame()
export['id'] = test_df['id']
export['target'] = test_df['target']
export = export.set_index('id')
export.to_csv('submit.csv')
#files.download('submit.csv')